In [1]:
import glob
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imutils

## Extract bounding box disc and cup optic from the same image
### REFUGE DATASET

In [2]:
def bounding_box_mask(path_img): 
    # 0 cup; 1 disc;
#     thresh = [127, 50] # máscaras do disco e escavação em uma única imagem
    thresh = [50]
    
    img_name = []
    yolo_boxes = []
    
    for i in range(len(thresh)): 
        mask_image = cv2.imread(path_img)
        file = path_img.split("\\")[-1][:-4]
        img_name.append(file)
        mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)
        mask_image = abs(255-mask_image)


        # Convert to binary image using thresholding
        binary_image = cv2.threshold(mask_image, thresh[i], 255, cv2.THRESH_BINARY)[1]  # cup optic

        # Find contours in the binary image
        contours = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = imutils.grab_contours(contours)

        # Filter contours based on area and aspect ratio
        bounding_boxes = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            x, y, w, h = cv2.boundingRect(cnt)
            bounding_boxes.append((x, y, w, h))
            
            cv2.rectangle(mask_image,(x,y),(x+w,y+h),(255,255,255),2)
            
#         # Plot rectangle around the masks
#         plt.imshow(mask_image)
#         plt.show()
            
        # Convert bounding box coordinates to YOLOv5 format
        image_height, image_width = mask_image.shape[:2]
        
        for bbox in bounding_boxes:
            x, y, w, h = bbox
            x_center = (x + w / 2) / image_width
            y_center = (y + h / 2) / image_height
            width = w / image_width
            height = h / image_height

            yolo_boxes.append((i, x_center, y_center, width, height))  # Class label is 0 for all boxes
    
    return yolo_boxes, img_name



In [7]:
def save_bnd_box(path, info, output):
    if info == 'test' or info == 'train':
        # path
        # - Class 1
        # -- imgs
        # - Class 2
        # -- imgs
        
        labels_ = os.listdir(path)
        for j, label in enumerate(labels_): # when folders are separated into glaucoma and normal
            cur_path = path + "/" + label 

            for image_path in glob.glob(cur_path + "/*"): 
                file = image_path.split("\\")[-1][:-4]
                print(file)
                yolo_boxes, img_name = bounding_box_mask(image_path)
                
                # Save bounding boxes to a file in YOLOv5 format
                with open(output + file + '.txt', 'w') as f:
                    for box in yolo_boxes:
                        f.write(f'{box[0]} {box[1]:.6f} {box[2]:.6f} {box[3]:.6f} {box[4]:.6f}\n')

    elif info == 'valid':
        # path:
        # --imgs
        
        for image_path in glob.glob(path + "/*"): # if all images in the same folder
            file = image_path.split("\\")[-1][:-4]
            print(file)
            yolo_boxes, img_name = bounding_box_mask(image_path)


            # Save bounding boxes to a file in YOLOv5 format
            with open(output + file + '.txt', 'w') as f:
                for box in yolo_boxes:
                    f.write(f'{box[0]} {box[1]:.6f} {box[2]:.6f} {box[3]:.6f} {box[4]:.6f}\n')

    return 



# train
save_bnd_box("D:/Glaucoma/Banco_de_dados/REFUGE/Annotation-Training400/Annotation-Training400/Disc_Cup_Masks", 'train', 'REFUGE/train/')
# # valid
save_bnd_box("D:/Glaucoma/Banco_de_dados/REFUGE/REFUGE-Validation400/REFUGE-Validation400-GT/Disc_Cup_Masks", 'valid', 'REFUGE/train/')
# test
save_bnd_box('D:/Glaucoma/Banco_de_dados/REFUGE/REFUGE-Test-GT/Disc_Cup_Masks', 'test', 'REFUGE/test/')

g0001
g0002
g0003
g0004
g0005
g0006
g0007
g0008
g0009
g0010
g0011
g0012
g0013
g0014
g0015
g0016
g0017
g0018
g0019
g0020
g0021
g0022
g0023
g0024
g0025
g0026
g0027
g0028
g0029
g0030
g0031
g0032
g0033
g0034
g0035
g0036
g0037
g0038
g0039
g0040
n0001
n0002
n0003
n0004
n0005
n0006
n0007
n0008
n0009
n0010
n0011
n0012
n0013
n0014
n0015
n0016
n0017
n0018
n0019
n0020
n0021
n0022
n0023
n0024
n0025
n0026
n0027
n0028
n0029
n0030
n0031
n0032
n0033
n0034
n0035
n0036
n0037
n0038
n0039
n0040
n0041
n0042
n0043
n0044
n0045
n0046
n0047
n0048
n0049
n0050
n0051
n0052
n0053
n0054
n0055
n0056
n0057
n0058
n0059
n0060
n0061
n0062
n0063
n0064
n0065
n0066
n0067
n0068
n0069
n0070
n0071
n0072
n0073
n0074
n0075
n0076
n0077
n0078
n0079
n0080
n0081
n0082
n0083
n0084
n0085
n0086
n0087
n0088
n0089
n0090
n0091
n0092
n0093
n0094
n0095
n0096
n0097
n0098
n0099
n0100
n0101
n0102
n0103
n0104
n0105
n0106
n0107
n0108
n0109
n0110
n0111
n0112
n0113
n0114
n0115
n0116
n0117
n0118
n0119
n0120
n0121
n0122
n0123
n0124
n0125
n0126
n012

## extract bounding box from many mask to only image
### Dataset 

In [ ]:
def save_bnd_box_retinopathy(path):
    # label encoding
    def label_encoding(x):
        labels = {'hard_exudates':0, 'hemorrhages':1, 'microaneurysms':2, 'soft_exsudates':3}
        return labels[x]


    labels_ = os.listdir(path)
    for j, label in enumerate(labels_): # when folders are separated into glaucoma and normal
        cur_path = path + "/" + label 


        for image_path in glob.glob(cur_path + "/*"): 

            bounding_boxes = []
            yolo_boxes = []

            print(image_path)
            img = cv2.imread(image_path)
            file = image_path.split("\\")[-1][:-4]
            img_name.append(file)

            # convert to grayscale
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

            # threshold
            thresh = cv2.threshold(gray, 10, 255,cv2.THRESH_BINARY)[1]

            # get contours
            result = img.copy()
            contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            contours = imutils.grab_contours(contours)

            for cntr in contours:
                x,y,w,h = cv2.boundingRect(cntr)
                bounding_boxes.append((x, y, w, h))
                cv2.rectangle(thresh, (x, y), (x+w, y+h), (255, 255, 255), 2)
#             print(bounding_boxes)
    #         fig = plt.figure(figsize = (6, 4))
    #         ax  = fig.add_subplot(111)
    #         ax.imshow(thresh.copy(), cmap='gray')

            # Convert bounding box coordinates to YOLOv5 format
            image_height, image_width = mask_image.shape[:2]

            for bbox in bounding_boxes:
                x, y, w, h = bbox
                x_center = (x + w / 2) / image_width
                y_center = (y + h / 2) / image_height
                width = w / image_width
                height = h / image_height

                yolo_boxes.append((label_encoding(label), x_center, y_center, width, height))  # Class label is 0 for all boxes

                
                
            output_path = 'C:/Users/Debora/Documents/YOLO/1_data_preparation/data_dr/image/'
            # Save bounding boxes to a file in YOLOv5 format
            with open(output_path + file + '.txt', 'a') as f:
                for box in yolo_boxes:
                    f.write(f'{box[0]} {box[1]:.6f} {box[2]:.6f} {box[3]:.6f} {box[4]:.6f}\n')
                    
                    
            # Remove duplicate
            with open(output_path + file + '.txt', 'r') as input_file:
                lines = input_file.readlines()
                # Remove duplicates from the list of lines
                unique_lines = sorted(list(set(lines)))
                # Open the output file and write the unique lines to it
                with open(output_path + file + '.txt', 'w') as output_file:
                    output_file.writelines(unique_lines)
                    
    return 
        


    
# path mask 
# -class 1
# --mask1 (mask same name to image)
# -class 2
# --maks2

save_bnd_box_retinopathy("C:/Users/Debora/Documents/YOLO/1_data_preparation/data_dr/masks")